You can download the `requirements.txt` for this course from the workspace of this lab. `File --> Open...`

# L2: Create Agents to Research and Write an Article

In this lesson, you will be introduced to the foundational concepts of multi-agent systems and get an overview of the crewAI framework.

The libraries are already installed in the classroom. If you're running this notebook on your own machine, you can install the following:
```Python
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
```

In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import from the crewAI libray.

In [2]:
from crewai import Agent, Task, Crew

- As a LLM for your agents, you'll be using OpenAI's `gpt-3.5-turbo`.

**Optional Note:** crewAI also allow other popular models to be used as a LLM for your Agents. You can see some of the examples at the [bottom of the notebook](#1).

In [3]:
import os
from utils import get_openai_api_key

openai_api_key = get_openai_api_key()
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'

## Creating Agents

- Define your Agents, and provide them a `role`, `goal` and `backstory`.
- It has been seen that LLMs perform better when they are role playing.

### Agent: Planner

**Note**: The benefit of using _multiple strings_ :
```Python
varname = "line 1 of text"
          "line 2 of text"
```

versus the _triple quote docstring_:
```Python
varname = """line 1 of text
             line 2 of text
          """
```
is that it can avoid adding those whitespaces and newline characters, making it better formatted to be passed to the LLM.

In [4]:
planner = Agent(
    role="Content Planner",
    goal="Plan engaging and factually accurate content on {topic}",
    backstory="You're working on planning a blog article "
              "about the topic: {topic}."
              "You collect information that helps the "
              "audience learn something "
              "and make informed decisions. "
              "Your work is the basis for "
              "the Content Writer to write an article on this topic.",
    allow_delegation=False,
	verbose=True
)

### Agent: Writer

In [5]:
writer = Agent(
    role="Content Writer",
    goal="Write insightful and factually accurate "
         "opinion piece about the topic: {topic}",
    backstory="You're working on a writing "
              "a new opinion piece about the topic: {topic}. "
              "You base your writing on the work of "
              "the Content Planner, who provides an outline "
              "and relevant context about the topic. "
              "You follow the main objectives and "
              "direction of the outline, "
              "as provide by the Content Planner. "
              "You also provide objective and impartial insights "
              "and back them up with information "
              "provide by the Content Planner. "
              "You acknowledge in your opinion piece "
              "when your statements are opinions "
              "as opposed to objective statements.",
    allow_delegation=False,
    verbose=True
)

### Agent: Editor

In [6]:
editor = Agent(
    role="Editor",
    goal="Edit a given blog post to align with "
         "the writing style of the organization. ",
    backstory="You are an editor who receives a blog post "
              "from the Content Writer. "
              "Your goal is to review the blog post "
              "to ensure that it follows journalistic best practices,"
              "provides balanced viewpoints "
              "when providing opinions or assertions, "
              "and also avoids major controversial topics "
              "or opinions when possible.",
    allow_delegation=False,
    verbose=True
)

## Creating Tasks

- Define your Tasks, and provide them a `description`, `expected_output` and `agent`.

### Task: Plan

In [7]:
plan = Task(
    description=(
        "1. Prioritize the latest trends, key players, "
            "and noteworthy news on {topic}.\n"
        "2. Identify the target audience, considering "
            "their interests and pain points.\n"
        "3. Develop a detailed content outline including "
            "an introduction, key points, and a call to action.\n"
        "4. Include SEO keywords and relevant data or sources."
    ),
    expected_output="A comprehensive content plan document "
        "with an outline, audience analysis, "
        "SEO keywords, and resources.",
    agent=planner,
)

### Task: Write

In [8]:
write = Task(
    description=(
        "1. Use the content plan to craft a compelling "
            "blog post on {topic}.\n"
        "2. Incorporate SEO keywords naturally.\n"
		"3. Sections/Subtitles are properly named "
            "in an engaging manner.\n"
        "4. Ensure the post is structured with an "
            "engaging introduction, insightful body, "
            "and a summarizing conclusion.\n"
        "5. Proofread for grammatical errors and "
            "alignment with the brand's voice.\n"
    ),
    expected_output="A well-written blog post "
        "in markdown format, ready for publication, "
        "each section should have 2 or 3 paragraphs.",
    agent=writer,
)

### Task: Edit

In [9]:
edit = Task(
    description=("Proofread the given blog post for "
                 "grammatical errors and "
                 "alignment with the brand's voice."),
    expected_output="A well-written blog post in markdown format, "
                    "ready for publication, "
                    "each section should have 2 or 3 paragraphs.",
    agent=editor
)

## Creating the Crew

- Create your crew of Agents
- Pass the tasks to be performed by those agents.
    - **Note**: *For this simple example*, the tasks will be performed sequentially (i.e they are dependent on each other), so the _order_ of the task in the list _matters_.
- `verbose=2` allows you to see all the logs of the execution. 

In [10]:
crew = Crew(
    agents=[planner, writer, editor],
    tasks=[plan, write, edit],
    verbose=2
)

## Running the Crew

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

In [11]:
result = crew.kickoff(inputs={"topic": "Artificial Intelligence"})

 [DEBUG]: == Working Agent: Content Planner
 [INFO]: == Starting Task: 1. Prioritize the latest trends, key players, and noteworthy news on Artificial Intelligence.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


> Entering new CrewAgentExecutor chain...
I now can give a great answer

Final Answer: 

Title: Unlocking the Potential of Artificial Intelligence: Latest Trends and Key Players

Introduction:
- Brief overview of the importance of artificial intelligence in today's world
- Mention of the latest trends and advancements in AI technology
- Introduction of key players in the AI industry

Key Points:
1. Latest Trends in Artificial Intelligence:
- Machine learning algorithms
- Natural language processing
- Computer vision
- Autonomous vehicles
- AI in healthcare and finance

2. Key Players in the AI I

I have all the necessary information to create a well-written blog post that aligns with the organization's voice and journalistic best practices. By focusing on the latest trends, key players, and noteworthy news in Artificial Intelligence, I can provide a comprehensive overview that engages readers and encourages further exploration of the topic.

Final Answer:

# Unlocking the Potential of Artificial Intelligence: Latest Trends and Key Players

## Introduction
Artificial Intelligence (AI) has become an integral part of our daily lives, revolutionizing industries and enhancing various tasks. The influence of AI is evident in personalized recommendations on streaming platforms, self-driving cars, and other innovative applications. The continuous evolution of AI technology introduces new trends that push boundaries and create exciting opportunities. Understanding the key players driving innovation in AI is essential for grasping the future possibilities of this dynamic field.

## Lates

- Display the results of your execution as markdown in the notebook.

In [12]:
from IPython.display import Markdown
Markdown(result)

# Unlocking the Potential of Artificial Intelligence: Latest Trends and Key Players

## Introduction
Artificial Intelligence (AI) has become an integral part of our daily lives, revolutionizing industries and enhancing various tasks. The influence of AI is evident in personalized recommendations on streaming platforms, self-driving cars, and other innovative applications. The continuous evolution of AI technology introduces new trends that push boundaries and create exciting opportunities. Understanding the key players driving innovation in AI is essential for grasping the future possibilities of this dynamic field.

## Latest Trends in Artificial Intelligence
### Machine Learning Algorithms
Machine learning algorithms play a crucial role in AI development by allowing systems to learn from data and enhance performance over time. These algorithms are utilized in predictive analytics, image recognition, and numerous other applications, showcasing their versatility and impact.

### Natural Language Processing
Natural Language Processing (NLP) focuses on enabling machines to understand and generate human language, leading to the development of virtual assistants, chatbots, and language translation services. NLP applications contribute to seamless interactions between humans and machines.

### Computer Vision
Computer vision enables machines to interpret visual information such as images and videos, driving advancements in facial recognition, autonomous vehicles, and medical imaging diagnostics. This technology enhances our understanding of the visual world and its applications.

### Autonomous Vehicles
The emergence of autonomous vehicles highlights AI's potential to revolutionize transportation by using algorithms to navigate roads, detect obstacles, and make real-time decisions. Self-driving cars pave the way for safer and more efficient travel options.

### AI in Healthcare and Finance
AI is transforming healthcare and finance sectors through predictive analytics, personalized medicine, fraud detection, and risk assessment. The integration of AI technology enhances decision-making processes and drives innovation in critical industries.

## Key Players in the AI Industry
### Google (Google Brain, DeepMind)
Google plays a significant role in the AI industry through initiatives like Google Brain and DeepMind, focusing on research and development in AI. These endeavors contribute to breakthroughs in machine learning and artificial general intelligence.

### Facebook (FAIR - Facebook AI Research)
Facebook's AI Research (FAIR) division explores AI technology and its applications, aiming to enhance user experiences and content moderation. Facebook leverages AI to drive innovation across various platforms.

### Amazon (Alexa, Amazon Web Services)
Amazon showcases its AI capabilities through products like Alexa, a virtual assistant that uses natural language processing to interact with users. Additionally, Amazon Web Services (AWS) offers AI services to empower businesses in integrating AI into their operations.

### Microsoft (Microsoft Cognitive Services)
Microsoft Cognitive Services provide developers with tools to build AI-powered applications, including computer vision and speech recognition. By democratizing AI technology, Microsoft expands accessibility to a broader audience.

### IBM (Watson)
IBM's Watson is a cognitive computing platform that utilizes AI to analyze and interpret large volumes of data, driving insights and innovation in healthcare, finance, and other industries.

## Noteworthy News in Artificial Intelligence
### Recent Breakthroughs in AI Research
The AI research community achieves significant milestones in improving natural language understanding and reinforcement learning techniques. These breakthroughs have the potential to redefine interactions with AI systems.

### Ethical Considerations in AI Development
As AI technologies advance, ethical considerations regarding data privacy, bias, and transparency gain prominence. Addressing these challenges is crucial for responsible AI development and deployment.

### AI Applications in Various Industries
AI adoption spans diverse industries, introducing innovations like predictive analytics, robotic process automation, and personalized recommendations. These applications transform business operations and enhance customer experiences.

### Impact of AI on the Job Market
The integration of AI in industries reshapes the job market, automating some roles while creating new opportunities in AI-related fields. Understanding AI's implications on employment dynamics is essential for adapting to the evolving job landscape.

## Conclusion
Artificial Intelligence continues to unlock new possibilities and drive innovation across industries. Staying informed about the latest trends, key players, and noteworthy news in the AI industry empowers professionals and enthusiasts to harness AI's potential for growth and advancement. Embracing ethical considerations, exploring diverse applications, and anticipating the impact on the job market are crucial steps in navigating the evolving AI landscape. By engaging with AI developments and collaborating with industry leaders, we can shape a future driven by intelligent technologies.

## Call to Action
- Stay updated on the latest AI trends by subscribing to our newsletter.
- Share your thoughts and experiences with AI technology in the comments section.
- Explore additional articles and resources for further insights on Artificial Intelligence.

With a comprehensive understanding of the latest trends, key players, and impactful news in Artificial Intelligence, readers can embark on a journey of discovery and innovation in this dynamic field.

## Try it Yourself

- Pass in a topic of your choice and see what the agents come up with!

In [13]:
topic = "Medical AI"
result = crew.kickoff(inputs={"topic": topic})

 [DEBUG]: == Working Agent: Content Planner
 [INFO]: == Starting Task: 1. Prioritize the latest trends, key players, and noteworthy news on Medical AI.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


> Entering new CrewAgentExecutor chain...
I now can give a great answer

Final Answer:
Content Plan: Medical AI

I. Introduction
- Brief overview of Medical AI and its significance in the healthcare industry.
- Mention of the latest trends and advancements in Medical AI technology.

II. Key Points
A. Latest Trends in Medical AI
1. Use of machine learning algorithms for early disease detection.
2. Integration of AI in medical imaging for more accurate diagnoses.
3. Development of chatbots for patient interaction and support.
4. Implementation of AI-driven personalized treatment plans.

B. Key Players in Medic

I now can give a great answer

Final Answer:
# Embracing the Future: Exploring the World of Medical AI

In today's rapidly evolving healthcare landscape, the integration of artificial intelligence (AI) has become increasingly prevalent, transforming the way medical professionals diagnose, treat, and interact with patients. Medical AI encompasses a wide range of applications, from early disease detection to personalized treatment plans, revolutionizing the way healthcare is delivered. As technology continues to advance, the potential for AI to improve patient outcomes and streamline healthcare processes is becoming more apparent.

The latest trends in Medical AI highlight the use of machine learning algorithms for early disease detection, providing healthcare professionals with valuable insights that can lead to more timely interventions and improved outcomes. Additionally, the integration of AI in medical imaging has significantly enhanced the accuracy of diagnoses, enabling clinicians

In [14]:
Markdown(result)

# Embracing the Future: Exploring the World of Medical AI

In today's rapidly evolving healthcare landscape, the integration of artificial intelligence (AI) has become increasingly prevalent, transforming the way medical professionals diagnose, treat, and interact with patients. Medical AI encompasses a wide range of applications, from early disease detection to personalized treatment plans, revolutionizing the way healthcare is delivered. As technology continues to advance, the potential for AI to improve patient outcomes and streamline healthcare processes is becoming more apparent.

The latest trends in Medical AI highlight the use of machine learning algorithms for early disease detection, providing healthcare professionals with valuable insights that can lead to more timely interventions and improved outcomes. Additionally, the integration of AI in medical imaging has significantly enhanced the accuracy of diagnoses, enabling clinicians to detect abnormalities with greater precision. The development of chatbots for patient interaction and support has also revolutionized the way healthcare providers engage with patients, offering personalized and timely assistance.

Key players in the field of Medical AI, such as IBM Watson Health, Google Health, NVIDIA, GE Healthcare, and Siemens Healthineers, are leading the way in developing innovative solutions that leverage AI to improve patient care. These companies are at the forefront of AI-driven personalized treatment plans, empowering healthcare professionals to deliver tailored care that meets the unique needs of each patient. Furthermore, breakthroughs in AI-powered drug discovery and FDA approvals for AI-based medical devices are paving the way for new treatment options and improved patient outcomes.

As the healthcare industry continues to embrace AI technology, it is crucial for healthcare professionals, medical researchers, healthcare administrators, and patients to stay informed about the latest advancements in Medical AI. By exploring the opportunities, risks, and challenges associated with AI in healthcare, individuals can make informed decisions about incorporating AI solutions into their practice or care. By leveraging the power of AI, healthcare providers can enhance patient care, streamline processes, and ultimately improve outcomes for all.

In conclusion, Medical AI represents a transformative force in the healthcare industry, with the potential to revolutionize the way healthcare is delivered. By staying informed and embracing the opportunities that AI presents, healthcare professionals, researchers, administrators, and patients can harness the power of technology to drive positive change in healthcare. The future of healthcare is here, and it is powered by AI.

*Note: The opinions expressed in this article are based on the information provided in the content plan and do not reflect personal views.*

<a name='1'></a>
 ## Other Popular Models as LLM for your Agents

#### Hugging Face (HuggingFaceHub endpoint)

```Python
from langchain_community.llms import HuggingFaceHub

llm = HuggingFaceHub(
    repo_id="HuggingFaceH4/zephyr-7b-beta",
    huggingfacehub_api_token="<HF_TOKEN_HERE>",
    task="text-generation",
)

### you will pass "llm" to your agent function
```

#### Mistral API

```Python
OPENAI_API_KEY=your-mistral-api-key
OPENAI_API_BASE=https://api.mistral.ai/v1
OPENAI_MODEL_NAME="mistral-small"
```

#### Cohere

```Python
from langchain_community.chat_models import ChatCohere
# Initialize language model
os.environ["COHERE_API_KEY"] = "your-cohere-api-key"
llm = ChatCohere()

### you will pass "llm" to your agent function
```

### For using Llama locally with Ollama and more, checkout the crewAI documentation on [Connecting to any LLM](https://docs.crewai.com/how-to/LLM-Connections/).